## Preprocessing

In [1]:
#  Install keras-tuner
! pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.6 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras_tuner as kt


In [3]:
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)
application_df.sample(10)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
8960,T3,Independent,C1200,Preservation,Trust,1,100000-499999,N,505588,1
2065,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,272976,1
7339,T3,CompanySponsored,C2100,Preservation,Trust,1,0,N,5000,0
26583,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,413525,1
20072,T3,CompanySponsored,C1000,Preservation,Trust,1,25000-99999,N,21748,0
6040,T3,CompanySponsored,C1000,Preservation,Trust,1,0,N,5000,1
1602,T3,Independent,C1000,Preservation,Trust,1,0,N,5000,1
23481,T3,CompanySponsored,C1000,Preservation,Trust,1,100000-499999,N,75817,0
8151,T3,CompanySponsored,C1000,Preservation,Trust,1,0,N,5000,0
24588,T9,Independent,C1000,ProductDev,Co-operative,1,5M-10M,N,1164016,1


In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_1 = class_counts.loc[class_counts > 1]
class_counts_1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Look at ASK_AMT value counts for binning
ask_amt_counts = application_df['ASK_AMT'].value_counts()
ask_amt_counts

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [12]:
# You may find it helpful to look at ASK_AMT value counts >1
ask_amt_1 = ask_amt_counts.loc[ask_amt_counts > 1]
ask_amt_1

5000      25398
10478         3
15583         3
63981         3
6725          3
          ...  
100000        2
8181          2
24746         2
45793         2
240780        2
Name: ASK_AMT, Length: 152, dtype: int64

In [13]:
# Choose a cutoff value and create a list of ask_amt to be replaced
# use the variable name `ask_amt_to_replace`
ask_amt_to_replace = list(ask_amt_counts[ask_amt_counts < 3].index)

# Replace in dataframe
for amt in ask_amt_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,"Other")

# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

5000     25398
Other     8889
15583        3
6725         3
10478        3
63981        3
Name: ASK_AMT, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
app_final = pd.get_dummies(application_df)

In [15]:
# Split our preprocessed data into our features and target arrays
y = app_final["IS_SUCCESSFUL"]
X = app_final.drop(["IS_SUCCESSFUL"], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size = 0.7)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model using kerastuner

In [17]:
# Create a method that creates a new Sequential model with hyperparameter options
number_input_features = len(X_train.values[0])
def create_model(hp):
  model = Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
  activation_choice = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
  model.add(Dense(units=hp.Int('first_units', min_value=1, max_value=50, step=5), activation=activation_choice, input_dim = number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
  for i in range(hp.Int("num_layers", 1, 5)):
    model.add(Dense(units=hp.Int(f'units_{i}', min_value=1, max_value=50, step=5), activation=activation_choice))
  model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
  model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
  return model

In [18]:
# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=25, hyperband_iterations=3)

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=25, validation_data=(X_test_scaled, y_test))

Trial 59 Complete [00h 01m 23s]
val_accuracy: 0.7294460535049438

Best val_accuracy So Far: 0.7324587106704712
Total elapsed time: 00h 31m 38s

Search: Running Trial #60

Value             |Best Value So Far |Hyperparameter
tanh              |relu              |activation
1                 |46                |first_units
4                 |3                 |num_layers
11                |11                |units_0
1                 |1                 |units_1
41                |26                |units_2
21                |31                |units_3
46                |26                |units_4
25                |25                |tuner/epochs
0                 |0                 |tuner/initial_epoch
0                 |0                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/25
751/751 [==============================] - 4s 4ms/step - loss: 0.6218 - accuracy: 0.6811 - val_loss: 0.5888 - val_accuracy: 0.7159
Epoch 2/25
751/751 [========================

In [ ]:
# Get top 3 model hyperparameters and print the values
for params in tuner.get_best_hyperparameters(3):
  print(params.values)

In [ ]:
# Get best model hyperparameters
best_model = tuner.get_best_hyperparameters(1)[0]
best_model.values

In [ ]:
# Evaluate the top 3 models against the test dataset
for model in tuner.get_best_models(3):
  model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
model.save("AlphabetSoupCharity_Optimization.h5")
